In [1]:
import pandas as pd

In [9]:
prices = pd.read_csv('prices.log', header=None, names=["Price", "Timestamp"])
predictions = pd.read_csv('predictions.log', header=None, names=["Price", "Timestamp"])

In [15]:
predictions.head()

,Price,Timestamp
0,1810.47,2025-04-02 17:44:50.649871700-05:00
1,1804.10,2025-04-02 17:47:22.515921900-05:00
2,1804.60,2025-04-02 17:49:54.522857200-05:00
3,1804.69,2025-04-02 17:52:27.046687800-05:00
4,1803.84,2025-04-02 17:54:59.449087100-05:00


In [12]:
# Assuming df_real and df_pred are the two DataFrames with real and predicted prices
prices['Timestamp'] = pd.to_datetime(prices['Timestamp']).dt.tz_convert ('EST')
predictions['Timestamp'] = pd.to_datetime(predictions['Timestamp']).dt.tz_convert ('EST')

# Sort both DataFrames by Timestamp
prices = prices.sort_values('Timestamp')
predictions = predictions.sort_values('Timestamp')

# Perform the merge with a 2-second tolerance
merged_df = pd.merge_asof(
    prices,
    predictions,
    on='Timestamp',
    tolerance=pd.Timedelta(seconds=2),
    direction='nearest',  # Options: 'backward', 'forward', 'nearest'
    suffixes=('_real', '_pred')
)

In [16]:
merged_df

,Price_real,Timestamp,Price_pred
0,1807.23,2025-04-02 18:41:17-05:00,NaN
1,1806.74,2025-04-02 18:41:32-05:00,NaN
2,1806.57,2025-04-02 18:41:47-05:00,NaN
3,1806.79,2025-04-02 18:42:02-05:00,NaN
4,1806.28,2025-04-02 18:42:17-05:00,NaN
...,...,...,...
235,1800.67,2025-04-02 19:40:54-05:00,NaN
236,1800.00,2025-04-02 19:41:10-05:00,NaN
237,1800.06,2025-04-02 19:41:25-05:00,NaN
238,1801.47,2025-04-02 19:41:40-05:00,NaN
